In [ ]:
import os
from glob import glob
import rasterio as rio
import numpy as np
from tqdm import tqdm,tqdm_notebook
from rasterio.plot import reshape_as_image,reshape_as_raster
import re
import pandas as pd
import json
import geopandas as gpd

import seaborn as sns
from rasterio.features import rasterize
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

from sklearn.cluster import MiniBatchKMeans
from rasterio.features import sieve
from rasterio.plot import reshape_as_image
from affine import Affine

In [ ]:
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance
from skimage.filters import threshold_otsu,threshold_multiotsu
def lee_filter(img, size):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

def compute_s1_rws(band):
    thresh = threshold_multiotsu(vh_lee,classes=3)
    regions = np.digitize(vh_lee, bins=thresh)
    s1_rws = np.where(regions==0,1,0).astype(rio.uint8)
    return s1_rws

In [ ]:
%%time

#Speckle filter (Lee's)
s1_files = glob(f'{os.path.abspath("..")}s1_2017/*.tif')

with rio.open(s1_files[-1]) as src:
    profile = src.profile.copy()
    img = src.read()
    vv,vh = img[0],img[1]
    vv[np.isnan(vv)]=0
    vh[np.isnan(vh)]=0
    
    vh_lee = lee_filter(vh,5)
    vv_lee = lee_filter(vv,5)
    s1_stack = np.array([vv_lee,vh_lee]).astype(rio.float32)
    
    #s1 rws 
    s1_rws = compute_s1_rws(vh_lee)
    s1_stack_sample = np.where(s1_rws==1,s1_stack,0)
    
    
    #export as geotiff
    profile.update({'dtype':out_img.dtype,'nodata':0,'count':1})
    with rio.open('test.tif','w',**profile) as dst:
        dst.write(np.array([out_img]) )
